In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import nltk
from nltk.corpus import stopwords
import spacy
from spacy.util import compounding
from spacy.util import minibatch
from spacy import displacy
import gc
import os

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
csv_collection = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        fullpath = os.path.join(dirname, filename)
        csv_collection.append(fullpath)

In [3]:
df = pd.read_csv(csv_collection.pop(), compression = 'gzip', index_col = 0)
for data in csv_collection:
    tmp = pd.read_csv(data, compression = 'gzip', index_col = 0)
    df = pd.concat([df, tmp], axis=0)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df.shape

(4213690, 17)

In [5]:
df.columns

Index(['userid', 'username', 'acctdesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreatedts', 'tweetid', 'tweetcreatedts',
       'retweetcount', 'text', 'hashtags', 'language', 'coordinates',
       'favorite_count', 'extractedts'],
      dtype='object')

In [6]:
sorted_tweets = df[['username', 'text', 'retweetcount', 'tweetid']].sort_values(by = 'retweetcount', ascending = False)

In [7]:
most_retweeted = sorted_tweets.iloc[0]
print(most_retweeted.text)

.@ZelenskyyUa's tv address to the Russian (!) people might be the most moving speech that I've ever seen in my entire life. The whole world needs to see, understand and share this crucial Ukrainian message.
#StandWithUkraine #Ukraine #Україна #Russia #Россия https://t.co/WoMOgqXTWX


<blockquote class="twitter-tweet"><p lang="en" dir="ltr">.<a href="https://twitter.com/ZelenskyyUa?ref_src=twsrc%5Etfw">@ZelenskyyUa</a>&#39;s tv address to the Russian (!) people might be the most moving speech that I&#39;ve ever seen in my entire life. The whole world needs to see, understand and share this crucial Ukrainian message.<a href="https://twitter.com/hashtag/StandWithUkraine?src=hash&amp;ref_src=twsrc%5Etfw">#StandWithUkraine</a> <a href="https://twitter.com/hashtag/Ukraine?src=hash&amp;ref_src=twsrc%5Etfw">#Ukraine</a> <a href="https://twitter.com/hashtag/%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B0?src=hash&amp;ref_src=twsrc%5Etfw">#Україна</a> <a href="https://twitter.com/hashtag/Russia?src=hash&amp;ref_src=twsrc%5Etfw">#Russia</a> <a href="https://twitter.com/hashtag/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F?src=hash&amp;ref_src=twsrc%5Etfw">#Россия</a> <a href="https://t.co/WoMOgqXTWX">pic.twitter.com/WoMOgqXTWX</a></p>&mdash; Patrick Moelleken 🇺🇦 (@PMoelleken) <a href="https://twitter.com/PMoelleken/status/1496941845812760577?ref_src=twsrc%5Etfw">February 24, 2022</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 

To find another Tweet use: twitter.com/anyuser/status/{tweetid}

In [8]:
#for example the above was found with
sorted_tweets.tweetid.iloc[0]

1500122026702422017